In [1]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function
import tensorflow as tf 
import keras
import gensim
from keras.engine.topology import Layer
from keras.layers import Dense, Input, Flatten, Dropout, BatchNormalization, Add, Flatten, LSTM, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Concatenate, AveragePooling1D, Lambda, Multiply, GlobalMaxPooling1D
from keras.layers import GRUCell, LSTMCell, RNN
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, RMSprop, SGD, Adadelta
from keras.models import Model, load_model
from keras import regularizers
from keras.preprocessing import sequence
import keras.backend as K
import numpy as np
from keras.datasets import imdb
import preprocessing
import Visualization as vis
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict, namedtuple
import logging
logger = logging.getLogger("wiki_word2vec")
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
import os.path as osp
import pickle

import importlib
importlib.reload(preprocessing)
importlib.reload(vis)

D:\Miniconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Miniconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


<module 'Visualization' from 'D:\\jupyter-workdir\\nlp\\bg_fusion\\Visualization.py'>

In [2]:
wordCounter = preprocessing.WordCounter()
wordCounter.words_list = pickle.load(open("E:/kaggle/avito/imdb_testset/aclImdb_v1/words_counter_list", "rb"))

In [3]:
total_word_count = len(wordCounter.words_list)
num_words = 30000
embed_size = 300
batch_size = 100
bins_count = 200
pad_maxlen = 100
sequence_len = 30
rnn_size = 512
rnn_layer_num = 3
epochs = 20
y_onehot = True

In [19]:
embedings = None
if False:
    WordVectorModel = namedtuple("WordVectorModel", ["wv"])
    wv = gensim.models.KeyedVectors.load_word2vec_format("E:/kaggle/avito/imdb_testset/dict2vec/dict2vec-vectors-dim300.vec.bin", binary=True)
    model = WordVectorModel(wv=wv)
    embedings, not_in_list = wordCounter.get_pretrain_embedding(model, num_words=num_words, size=embed_size)
    del wv

In [5]:
class DynamicRNN(Layer):
    def __init__(self, units, n_layers, input_keep_prob = 1.0, output_keep_prob = 1.0, hidden_state = None, reuse = True, unroll = False, **kwargs):
        super(DynamicRNN, self).__init__(**kwargs)
        self.units = units
        self.n_layers = n_layers
        self.input_keep_prob = input_keep_prob
        self.output_keep_prob = output_keep_prob
        self.hidden_state = hidden_state
        self.reuse = reuse
        self.unroll = unroll
        self.i_built = False
    
    def build(self, input_shape):
        self.cells = [tf.contrib.rnn.GRUCell(self.units, reuse=self.reuse) for _ in range(self.n_layers)]
        self.dropcells = [tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=self.input_keep_prob) for cell in self.cells]
        self.multicell = tf.contrib.rnn.MultiRNNCell(self.dropcells, state_is_tuple=False)
        self.multicell_wrapper = tf.contrib.rnn.DropoutWrapper(self.multicell, output_keep_prob=self.output_keep_prob)  
        super(DynamicRNN, self).build(input_shape)
    
    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0], input_shape[1], self.units)
        state_shape = (input_shape[0], self.multicell.state_size)
        return [output_shape, state_shape]
    
    def get_config(self):
        config = {"units": self.units, "n_layers": self.n_layers, "input_keep_prob": self.input_keep_prob, 
                  "output_keep_prob": self.output_keep_prob, "reuse": self.reuse, "unroll": self.unroll}
        base_config = super(DynamicRNN, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    def get_initial_state(self, inputs):
        if self.hidden_state is None:
            # build an all-zero tensor of shape (samples, output_dim)
            initial_state = K.zeros_like(inputs)  # (samples, timesteps, input_dim)
            initial_state = K.sum(initial_state, axis=(1, 2))  # (samples,)
            initial_state = K.expand_dims(initial_state)  # (samples, 1)
            return K.tile(initial_state, [1, self.multicell.state_size])
        else:
            return self.hidden_state
    
    def call(self, x):
        if self.unroll:
            sequence_len = x.shape[1].value
            state = self.get_initial_state(x)
            Yr = [] # [ SEQLEN, BATCHSIZE, INTERNALSIZE ]
            for unroll_step in range(sequence_len):
                cell_output, state = self.multicell_wrapper(x[:, unroll_step, :], state)
                Yr.append(cell_output)
            Yr = tf.convert_to_tensor(Yr)
            Yr = tf.transpose(Yr, [1, 0, 2])
            H = state # [ BATCHSIZE, INTERNALSIZE *  NLAYERS ]
        else:
            Yr, H = tf.nn.dynamic_rnn(self.multicell_wrapper, inputs=x, dtype=tf.float32, initial_state=self.get_initial_state(x))
        if not self.i_built:
            self.i_built = True
            self._trainable_weights.extend(self.multicell.weights)
        # Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
        # H:   # this is the last state in the sequence
        return [Yr, H]

In [6]:
def getdata(num_words=None):
    state = np.random.RandomState(0)
    X_train_pos = np.array(wordCounter.transform(["E:/kaggle/avito/imdb_testset/aclImdb_v1/train_pos.txt"], max_words=num_words))
    X_train_neg = np.array(wordCounter.transform(["E:/kaggle/avito/imdb_testset/aclImdb_v1/train_neg.txt"], max_words=num_words))
    y_train_pos = np.ones(X_train_pos.shape[0])
    y_train_neg = np.zeros(X_train_neg.shape[0])
    X_test_pos = np.array(wordCounter.transform(["E:/kaggle/avito/imdb_testset/aclImdb_v1/test_pos.txt"], max_words=num_words))
    X_test_neg = np.array(wordCounter.transform(["E:/kaggle/avito/imdb_testset/aclImdb_v1/test_neg.txt"], max_words=num_words))
    y_test_pos = np.ones(X_test_pos.shape[0])
    y_test_neg = np.zeros(X_test_neg.shape[0])
    X_train, y_train = np.concatenate([X_train_pos, X_train_neg]), np.concatenate([y_train_pos, y_train_neg])
    X_test, y_test = np.concatenate([X_test_pos, X_test_neg]), np.concatenate([y_test_pos, y_test_neg])
    train_permut = state.permutation(X_train.shape[0])
    test_permut = state.permutation(X_test.shape[0])
    
    X_train_unsup = np.array(wordCounter.transform(["E:/kaggle/avito/imdb_testset/aclImdb_v1/train_unsup.txt"], max_words=num_words))
    unsup_permut = state.permutation(X_train_unsup.shape[0])
    
    return (X_train[train_permut], y_train[train_permut]), (X_test[test_permut], y_test[test_permut]), X_train_unsup[unsup_permut]

In [7]:
def gather_texts(docs_list, n_words_per_unit = 20, padding_ch = 0):
    units = []
    for i, docs in enumerate(docs_list):
        print("processing doc-%s" % i)
        for doc in docs:
            units.extend(doc)
    return units

In [8]:
(X_train_o, y_train), (X_test_o, y_test), X_train_unsup_o = getdata(num_words=num_words)

In [9]:
X_all = gather_texts([X_train_o, X_test_o, X_train_unsup_o])

processing doc-0
processing doc-1
processing doc-2


In [10]:
# X_train = sequence.pad_sequences(X_train_o, maxlen=pad_maxlen, padding='post', truncating='post')
# X_test = sequence.pad_sequences(X_test_o, maxlen=pad_maxlen, padding='post', truncating='post')
# X_train_unsup = sequence.pad_sequences(X_train_unsup_o, maxlen=pad_maxlen, padding='post', truncating='post')

In [11]:
# X_train = X_train_o
# X_test = X_test_o
# X_train_unsup = X_train_unsup_o

In [12]:
def get_steps(raw_data, batch_size, sequence_size):
    data_len = len(raw_data)
    nb_batches = (data_len - 1) // (batch_size * sequence_size)
    return nb_batches

def rnn_minibatch_sequencer(raw_data, batch_size, sequence_size, nb_epochs):
    """
    Divides the data into batches of sequences so that all the sequences in one batch
    continue in the next batch. This is a generator that will keep returning batches
    until the input data has been seen nb_epochs times. Sequences are continued even
    between epochs, apart from one, the one corresponding to the end of raw_data.
    The remainder at the end of raw_data that does not fit in an full batch is ignored.
    :param raw_data: the training text
    :param batch_size: the size of a training minibatch
    :param sequence_size: the unroll size of the RNN
    :param nb_epochs: number of epochs to train on
    :return:
        x: one batch of training sequences
        y: on batch of target sequences, i.e. training sequences shifted by 1
        epoch: the current epoch number (starting at 0)
    """
    data = np.array(raw_data)
    data_len = data.shape[0]
    # using (data_len-1) because we must provide for the sequence shifted by 1 too
    nb_batches = (data_len - 1) // (batch_size * sequence_size)
    assert nb_batches > 0, "Not enough data, even for a single batch. Try using a smaller batch_size."
    rounded_data_len = nb_batches * batch_size * sequence_size
    xdata = np.reshape(data[0:rounded_data_len], [batch_size, nb_batches * sequence_size])
    ydata = np.reshape(data[1:rounded_data_len + 1], [batch_size, nb_batches * sequence_size])

    for epoch in range(nb_epochs):
        for batch in range(nb_batches):
            x = xdata[:, batch * sequence_size:(batch + 1) * sequence_size]
            y = ydata[:, batch * sequence_size:(batch + 1) * sequence_size]
            x = np.roll(x, -epoch, axis=0)  # to continue the text from epoch to epoch (do not reset rnn state!)
            y = np.roll(y, -epoch, axis=0)
            yield x, y

In [13]:
class SentenceGenerator(object):
    def __init__(self, X, y = None, batch_size = 125, bins_count=100, mode="train", onehot = False):
        self.batch_size = batch_size
        padding_mode = "random" if mode == "train" else "specific"
        self.pool = preprocessing.AutoPaddingInMemorySamplePool(X, chunk_size=batch_size, bins_count=bins_count, mode=padding_mode)
        self.y = y
        self.gen_mode = 0
        if self.y is not None:
            self.y = y[self.pool.sorted_indices]
        else:
            self.gen_mode = 1
        self.mode = mode
        self.y_indices_record = []
        self.onehot = onehot
        if self.onehot and self.y is not None:
            onehot_encoder = OneHotEncoder()
            self.y = self.y.reshape((-1, 1))
            self.y = onehot_encoder.fit_transform(self.y).toarray()
    
    def iter(self):
        return self
    
    def reset(self):
        self.pool.reset()
        self.y_indices_record = []
    
    def __next__(self):
        batch_samples = self.pool.__next__()
        if self.y is not None:
            if self.mode == "test":
                self.y_indices_record.extend(self.pool.chunk_indices_list)
            return batch_samples, self.y[self.pool.chunk_indices_list]
        else:
            n_samples, n_cols = batch_samples.shape
            # generate text target
            # got w0, predict w1
            y_batch_samples = np.roll(batch_samples, -1, axis=1)
            # drop last word
            batch_samples = batch_samples[:, 0: n_cols - 1]
            y_batch_samples = y_batch_samples[:, 0: n_cols - 1]
            return batch_samples, y_batch_samples
    
    def get_test_y(self, steps):
        if self.y is not None:
            return self.y[self.y_indices_record[: steps * self.batch_size]]
        else:
            return None
if False:
    SentGener_train_all = SentenceGenerator(batch_size=batch_size, X=np.concatenate([X_train, X_test, X_train_unsup]), y=None, mode="train", bins_count=bins_count, onehot=y_onehot)

In [14]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, mode="min"), 
    ModelCheckpoint(monitor="val_loss", filepath="E:/kaggle/avito/imdb_testset/tf_model/generator_net_imdb.hdf5", save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=5, min_lr=1e-5, verbose=1)
]

In [15]:
def nce_loss_for_keras(vocab_size, rnn_size, model_input, contrast_num_samples=5):
    nce_weights = tf.Variable(tf.truncated_normal([vocab_size, rnn_size], stddev=1.0 / np.sqrt(rnn_size)))
    nce_biases = tf.Variable(tf.zeros([vocab_size]))
    def get_nce_loss(y_true, y_pred):
        # y_true shape: (batch_size, sequence_len)
        # y_pred shape: (batch_size * sequence_len, rnn_size)
        y_true = K.reshape(y_true, [-1, 1]) # shape (batch_size * sequence_len, 1)
        nce_mean_loss = tf.reduce_mean(tf.nn.nce_loss(
                                             weights=nce_weights,
                                             biases=nce_biases,
                                             labels=y_true,
                                             inputs=y_pred,
                                             num_sampled=contrast_num_samples,
                                             num_classes=vocab_size))
        return nce_mean_loss
    
    def nce_accuracy(y_true, y_pred):
        # y_true shape: (batch_size, sequence_len)
        # y_pred shape: (batch_size * sequence_len, rnn_size)
        y_true = K.reshape(y_true, [-1]) # shape (batch_size * sequence_len, )
        word_logits = tf.matmul(y_pred, tf.transpose(nce_weights)) # shape (batch_size * sequence_len, vocab_size)
        word_logits = tf.nn.bias_add(word_logits, nce_biases) # shape (batch_size * sequence_len, vocab_size)
        word_logits = tf.argmax(word_logits, axis=-1) # shape (batch_size * sequence_len, )
        y_true = tf.cast(y_true, tf.int32) # float32 -> int32
        word_logits = tf.cast(word_logits, tf.int32) # int64 -> int32
        return tf.cast(tf.equal(y_true, word_logits), tf.float32)
        
    # model_input shape (batch_size * sequence_len, rnn_size)
    word_logits = tf.matmul(model_input, tf.transpose(nce_weights)) # shape (batch_size * sequence_len, vocab_size)
    word_logits = tf.nn.bias_add(word_logits, nce_biases) # shape (batch_size * sequence_len, vocab_size)
    word_logits = tf.argmax(word_logits, axis=-1) # shape (batch_size * sequence_len, 1)
    return get_nce_loss, nce_accuracy, word_logits

def reset_graph():
    tf.reset_default_graph()
    K.get_session().close()
    K.set_session(tf.Session())

In [16]:
# reset_graph()

In [17]:
def get_text_generator_net(num_words, embed_size = 100, rnn_size = 512, rnn_layer_num = 3, contrast_num_samples = 64, embedings = None):
    word_indices = Input(shape=[None], name="word_indices") # shape (batch_size, sequence_len)
    hidden_state = Input(shape=[rnn_layer_num * rnn_size], name="rnn_hidden_state") # shape (batch_size, rnn_layer_num * rnn_size)
    if embedings is None:
        word_embedding = Embedding(num_words, embed_size, name="word_embedding")(word_indices)
    else:
        word_embedding = Embedding(num_words, embed_size, weights=[embedings], name="word_embedding", trainable=False)(word_indices)
    x_flow = word_embedding # shape (batch_size, sequence_len, embed_size) (None, None, embed_size)
    rnn_outputs = DynamicRNN(units=rnn_size, n_layers=rnn_layer_num, input_keep_prob = 1.0, output_keep_prob = 0.8, hidden_state=hidden_state, reuse=tf.AUTO_REUSE)(x_flow)
    x_flow = rnn_outputs[0] # shape (batch_size, sequence_len, rnn_size)
    rnn_states = rnn_outputs[1] # shape (batch_size, rnn_size * rnn_layer_num)
    # x_flow = Lambda(lambda x: K.reshape(x, [-1, rnn_size]))(x_flow) # shape (batch_size * sequence_len, rnn_size)
    # nce_loss, accuracy, word_logits = nce_loss_for_keras(vocab_size=num_words, rnn_size=rnn_size, contrast_num_samples=contrast_num_samples, model_input=x_flow)
    x_flow = Dense(num_words, activation='softmax')(x_flow) # shape (batch_size, sequence_len, vocab_size)
    sgd = Adam(lr=1e-3)
    model = Model(inputs=[word_indices, hidden_state], outputs=x_flow)
    # model.word_logits = word_logits
    model.rnn_states = rnn_states
    model.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["acc"])
    generator_model = Model(inputs=[word_indices, hidden_state], outputs=[x_flow, rnn_states])
    return model, generator_model

In [20]:
generator_net_model, generator_model = get_text_generator_net(num_words=num_words, rnn_size=rnn_size, rnn_layer_num=rnn_layer_num, embed_size=embed_size, embedings=embedings)

In [21]:
generator_net_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_indices (InputLayer)    (None, None)              0         
_________________________________________________________________
word_embedding (Embedding)   (None, None, 300)         9000000   
_________________________________________________________________
dynamic_rnn_1 (DynamicRNN)   [(None, None, 512), (None 4397568   
_________________________________________________________________
dense_1 (Dense)              (None, None, 30000)       15390000  
Total params: 28,787,568
Trainable params: 28,787,568
Non-trainable params: 0
_________________________________________________________________


In [124]:
def generator_with_rnnstate(sentGener, batch_size, rnn_layer_num, rnn_size):
    rnnstate = np.zeros((batch_size, rnn_layer_num * rnn_size))
    while True:
        X, y = next(sentGener)
        # X shape (batch_size, seq_len)
        # Y shape (batch_size, seq_len)
        y = np.expand_dims(y, 2)
        # Y shape (batch_size, seq_len, 1)
        yield [X, rnnstate], y

In [125]:
callbacks[1].best = -np.inf
callbacks[2].best = -np.inf
generator = generator_with_rnnstate(rnn_minibatch_sequencer(X_all, batch_size, sequence_size=sequence_len, nb_epochs=epochs), batch_size, rnn_layer_num, rnn_size)
steps = get_steps(X_all, batch_size, sequence_size=batch_size)
generator_net_model.fit_generator(generator, steps_per_epoch=steps, epochs=epochs, shuffle=True, verbose=1)

Epoch 1/20
2732/2732 [==============================] - 882s 323ms/step - loss: 4.9475 - acc: 0.2094
Epoch 2/20
2732/2732 [==============================] - 883s 323ms/step - loss: 4.7085 - acc: 0.2242
Epoch 3/20
2732/2732 [==============================] - 880s 322ms/step - loss: 4.6321 - acc: 0.2301
Epoch 4/20
2732/2732 [==============================] - 882s 323ms/step - loss: 4.5701 - acc: 0.2345
Epoch 5/20
2732/2732 [==============================] - 882s 323ms/step - loss: 4.5276 - acc: 0.2376
Epoch 6/20
2732/2732 [==============================] - 880s 322ms/step - loss: 4.4971 - acc: 0.2394
Epoch 7/20
2732/2732 [==============================] - 883s 323ms/step - loss: 4.4734 - acc: 0.2414
Epoch 8/20
2732/2732 [==============================] - 881s 323ms/step - loss: 4.4496 - acc: 0.2433
Epoch 9/20
2732/2732 [==============================] - 885s 324ms/step - loss: 4.4345 - acc: 0.2439
Epoch 10/20
2732/2732 [==============================] - 881s 322ms/step - loss: 4.4184 - a

In [132]:
# generator_net_model.layers[2].reuse = True

In [26]:
generator_net_model.load_weights("E:/kaggle/avito/imdb_testset/tf_model/generator_net_imdb_type-A.hdf5")

In [22]:
wordCounter.i2w_dict = wordCounter.get_i2w_dictionary(num_words=num_words)
wordCounter.w2i_dict = wordCounter.get_w2i_dictionary(num_words=num_words)

In [23]:
def predict_text_sequence(model, start_word, max_seq_len, vocab_size, wordCounter, rnn_layer_num, rnn_size):
    start_word = start_word.strip().lower()
    i2w_dict = wordCounter.i2w_dict
    w2i_dict = wordCounter.w2i_dict
    assert start_word in w2i_dict, "%s not in word_dict" % start_word
    start_word_indice = np.zeros((1, 1), dtype=np.int32) # (batch_size, seq_len)
    start_word_indice[0, 0] = w2i_dict[start_word]
    rnn_state = np.zeros((1, rnn_layer_num * rnn_size))
    generated_sequence = [w2i_dict[start_word]]
    for step_i in range(max_seq_len):
        y, rnn_state = model.predict([start_word_indice, rnn_state])
        # y (batch_size, seq_len, vocab_size)
        # rnn_state (batch_size, rnn_size * rnn_layer_num)
        predict_word_index = np.argmax(y[0][0])
        start_word_indice[0, 0] = predict_word_index
        # rnn_state = K.get_session().run(model.rnn_states, feed_dict={model.get_input_at(0)[0]: start_word_indice, model.get_input_at(0)[1]: rnn_state})
        generated_sequence.append(predict_word_index)
    return wordCounter.reverse([generated_sequence], num_words=vocab_size, return_list=False)

In [34]:
predict_text_sequence(generator_model, "i", 200, num_words, wordCounter, rnn_layer_num, rnn_size)

["i ' m sure you ' ll be disappointed . i ' ve seen a lot of movies , but this one is just plain bad . i ' m not a big fan of the carpenters , but this movie is just plain bad . i don ' t know if it ' s a bad movie , but i ' m sure it ' s a great movie . i ' ve seen a lot of bad movies but i ' ve seen a lot of bad movies . i ' ve seen a lot of bad movies and i ' ve seen a lot of stinkers but this one is just plain bad . i ' ve seen worse movies in my time , but this one is just plain bad . i ' m not a big fan of the original but i ' ve seen better movies . i ' ve seen a lot of bad movies and i ' ve seen a lot of stinkers . i ' ve always liked the unk and the ghoulies . i ' ve always liked the original and the original . i ' ve seen a lot of"]